In [ ]:
!wget http://cb.lk/covid_19


--2022-05-31 08:23:42--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 172.67.187.81, 104.21.72.203, 2606:4700:3032::ac43:bb51, ...
Connecting to cb.lk (cb.lk)|172.67.187.81|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2022-05-31 08:23:42--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|172.67.187.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2022-05-31 08:23:43--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2022-0

In [ ]:
!unzip covid_19

Archive:  covid_19
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [Y]
error:  invalid response [[Y]]
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [Y]es
error:  invalid response [[Y]es]
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
replace CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [Y]ES
error:  invalid response [[Y]ES]
replace CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [y]es
error:  invalid response [[y]es]
replace CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
replace CovidDataset/Val/Covid/85E52EB3-56E9-4D67-8

In [ ]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

In [ ]:
import numpy as np

import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image


In [ ]:
#train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)



In [ ]:
train_data = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (256,256),
    batch_size = 16,
    class_mode = 'binary')

Found 224 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
test_datagen = image.ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    'CovidDataset/Val',
    target_size = (256,256),
    batch_size = 16,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [ ]:
#Training model
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size=(3,3),activation='relu',input_shape=(256,256,3)))

model.add(Conv2D(filters= 64,kernel_size = (3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))
 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128 ,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 conv2d_5 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [ ]:
hist = model.fit(
    train_data,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = test_data,
    validation_steps=2
)

Epoch 1/10
8/8 [==============================] - 7s 834ms/step - loss: 0.8267 - accuracy: 0.5938 - val_loss: 0.6745 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 6s 770ms/step - loss: 0.5878 - accuracy: 0.7109 - val_loss: 0.4741 - val_accuracy: 0.8750
Epoch 3/10
8/8 [==============================] - 6s 804ms/step - loss: 0.4114 - accuracy: 0.7969 - val_loss: 0.5333 - val_accuracy: 0.9375
Epoch 4/10
8/8 [==============================] - 6s 828ms/step - loss: 0.4076 - accuracy: 0.8359 - val_loss: 0.3235 - val_accuracy: 0.9688
Epoch 5/10
8/8 [==============================] - 7s 841ms/step - loss: 0.2590 - accuracy: 0.8984 - val_loss: 0.1659 - val_accuracy: 0.9688
Epoch 6/10
8/8 [==============================] - 6s 754ms/step - loss: 0.2810 - accuracy: 0.8750 - val_loss: 0.2370 - val_accuracy: 0.9375
Epoch 7/10
8/8 [==============================] - 6s 782ms/step - loss: 0.1124 - accuracy: 0.9531 - val_loss: 0.1046 - val_accuracy: 0.9688
Epoch 8/10
8/8 [====

In [ ]:
model.evaluate(train_data)

14/14 [==============================] - 8s 538ms/step - loss: 0.0887 - accuracy: 0.9821


[0.08870819956064224, 0.9821428656578064]

In [ ]:
model.evaluate(test_data)

4/4 [==============================] - 2s 390ms/step - loss: 0.0644 - accuracy: 0.9833


[0.06442607939243317, 0.9833333492279053]

In [ ]:
path="/content/CovidDataset/Val/Covid/16654_1_1.png"
img=image.load_img(path,target_size=(256,256))
img=image.img_to_array(img)/255
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
predictions = (model.predict(img) > 0.5).astype("int32")

In [ ]:
if predictions==1:
  print("normal")
else:
  print("covid")

covid


# New Section

# New Section